In [1]:
from sympy import Symbol, diff
import numpy as np 

In [2]:
n_variaveis = 2

x_1 = Symbol('x_1')
x_2 = Symbol('x_2')

funcao = (x_1**2 + x_2 - 11)**2 + (x_1 + x_2**2 - 7)**2
funcao

(x_1 + x_2**2 - 7)**2 + (x_1**2 + x_2 - 11)**2

In [3]:
def calcula_gradiente(funcao, variaveis):
    return [diff(funcao, variavel) for variavel in variaveis]

In [4]:
gradiente = calcula_gradiente(funcao, [x_1, x_2])
gradiente

[4*x_1*(x_1**2 + x_2 - 11) + 2*x_1 + 2*x_2**2 - 14,
 2*x_1**2 + 4*x_2*(x_1 + x_2**2 - 7) + 2*x_2 - 22]

In [5]:
def substitui_variaveis_funcao(funcao, variaveis, ponto):
    return funcao.subs(dict(zip(variaveis, ponto)))

def substitui_variaveis_gradiente(gradiente, variaveis, ponto):
    return np.array([gradiente_i.subs(dict(zip(variaveis, ponto))) for gradiente_i in gradiente])

![Armijo's Rule](./img/alg_busca_armijo.png)

In [39]:
def busca_de_armijo(funcao: any, gradiente: any, variaveis: list, ponto: np.array, d: np.array, gama: float, eta: float):
  """
  Parametros:
    - funcao: função a ser minimizada
    - gradiente: gradiente da função a ser minimizada
    - variaveis: lista de variáveis da função
    - ponto: ponto inicial
    - d: direção de descida
    - gama: fator de redução do passo
    - eta: fator de ???
  
  Retorna:
    - t: passo ótimo
    - iteracoes: número de iterações
  """
  t = 1
  iteracoes = 0
  while substitui_variaveis_funcao(funcao, variaveis, ponto + t*d) > substitui_variaveis_funcao(funcao, variaveis, ponto) + eta*t*np.dot(substitui_variaveis_gradiente(gradiente, variaveis, ponto), d):
    t = gama*t
    iteracoes += 1

  return t, iteracoes

In [42]:
from sympy import symbols

# Define the variables x1 and x2
x1, x2 = symbols('x1 x2')

# Define the function
f_x = (1/2) * (x1 - 2)**2 + (x2 - 1)**2
f_x
print(f_x)
# Define the gradient
grad_f_x = [f_x.diff(x1), f_x.diff(x2)]
print(grad_f_x)



0.5*(x1 - 2)**2 + (x2 - 1)**2
[1.0*x1 - 2.0, 2*x2 - 2]


In [46]:
from sympy import symbols

# Define the variables x1 and x2
variaveis = list(symbols('x1 x2'))
print(variaveis)
x1, x2 = variaveis

# Define the function
f_x = (1/2) * (x1 - 2)**2 + (x2 - 1)**2
f_x
print(f_x)

# Define the gradient
grad_f_x = calcula_gradiente(f_x, variaveis)
print(grad_f_x)



[x1, x2]
0.5*(x1 - 2)**2 + (x2 - 1)**2
[1.0*x1 - 2.0, 2*x2 - 2]


In [48]:
ponto_inicial = np.array([1, 0])
d = np.array([3, 1])
eta = 0.25
gama = 0.8

passo_otimo, n_iteracoes = busca_de_armijo(f_x, grad_f_x, variaveis, ponto_inicial, d, gama, eta)